In [1]:
!pip install pytorch-nlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 3.1 MB/s eta 0:00:00


In [2]:
import codecs
import pandas as pd

import torch
import torchtext
import numpy as np
import torchtext
import pandas
import collections
import collections
from torchnlp import *
from torch.optim import lr_scheduler
import shutil

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### Перевод в csv

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.DataFrame( columns=['lable', 'text'])
with codecs.open( "/content/drive/MyDrive/works/Пушкин Александр. Капитанская дочка.txt", "r", "utf_8" ) as file:
    lines=file.readlines()
    for line in lines:
        # line = file.readline()
        if (line != '\n') and (line !="***\n") and (line !="...\n"):
            line=line.replace('\n', '', 1)
            # print(line,type(line))
            df.loc[len(df.index)]=[1 , line]


In [6]:
df.to_csv("/content/drive/MyDrive/works/Pushkin.csv")

## Загрузка датасета с токенизацией по словам

In [7]:
def load_dataset_HP(ngrams=1,min_freq=1):
    global vocab, tokenizer
    train_dataset = pandas.read_csv('/content/drive/MyDrive/works/Pushkin.csv')["text"]
    train_dataset = list(train_dataset)
    counter = collections.Counter()
    for line in train_dataset:
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(str(line)),ngrams=ngrams))
    vocab = torchtext.vocab.vocab(counter, min_freq=min_freq)
    return train_dataset,vocab

tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

train_dataset,vocab_slov=load_dataset_HP()
vocab_size_slov = len(vocab_slov)

### Посимвольная токенизация

In [8]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for line in train_dataset:
    counter.update(char_tokenizer(str(line)))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)



### Функции Кодирования токенов

In [9]:
stoi_hash = {}
def encode(x,voc=None,unk=0,tokenizer=tokenizer):
    global stoi_hash
    v = vocab if voc is None else voc
    if v in stoi_hash.keys():
        stoi = stoi_hash[v]
    else:
        stoi = v.get_stoi()
        stoi_hash[v]=stoi
    return [stoi.get(s,unk) for s in tokenizer(x)]

In [10]:
## Ф-ии Кодирование токинов по словам
def enc_slov(x):
    return torch.LongTensor(encode(x,voc=vocab_slov,tokenizer=tokenizer))

In [11]:
## Ф-ии Кодирование токинов по символам
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

### Функция Создания Батча в эпохе

In [12]:
def get_batch(s,nchars=10):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

In [13]:
def get_batch_slov(s,nchars=10):
    leng = len(enc_slov(s))
    inps = torch.zeros(leng-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(leng-nchars,nchars,dtype=torch.long,device=device)
    for i in range(leng-nchars):
        inps[i] = enc_slov(s)[i:i+nchars]
        outs[i] = enc_slov(s)[i+1:i+nchars+1]
    return inps,outs

## Генератор следующией послдовательности символов/слов

In [14]:
def generate(net,size=100,start='Сегодня '):
    chars = list(start)
    out = net(enc(chars).view(1,-1).to(device))
    for i in range(size):
        nc = torch.argmax(out[0][-1])
        chars.append(vocab.get_itos()[nc])
        out = net(nc.view(1,-1))
    return ''.join(chars)

def generate_slov(net,size=100,start='Сегодня '):
    chars = start
    out = net(enc_slov(start).view(1,-1).to(device))
    for i in range(size):
        nc = torch.argmax(out[0][-1])
        chars+= " " + (vocab_slov.get_itos()[nc])
        out = net(nc.view(1,-1))
    return ''.join(chars)

## Мягкая генерация

In [15]:
def generate_soft(net,size=100,start='Сегодня маг ',temperature=1.0,enc=enc):
    chars = list(start)
    out = net(enc(chars).view(1,-1).to(device))
    for i in range(size):
        #nc = torch.argmax(out[0][-1])
        out_dist = out[0][-1].div(temperature).exp()
        nc = torch.multinomial(out_dist,1)[0]
        chars.append(vocab.get_itos()[nc])
        out = net(nc.view(1,-1))
    return ''.join(chars)

def generate_soft_slov(net,size=100,start='Сегодня маг ',temperature=1.0,enc=enc):
    chars = start
    out = net(enc_slov(chars).view(1,-1).to(device))
    for i in range(size):
        #nc = torch.argmax(out[0][-1])
        out_dist = out[0][-1].div(temperature).exp()
        nc = torch.multinomial(out_dist,1)[0]
        chars += " " + vocab_slov.get_itos()[nc]
        out = net(nc.view(1,-1))
    return ''.join(chars)


## Ф-ия обучения

In [16]:
def train(net,train_dataset,val_dataset,num_epochs, optimizer, scheduler, get_batch, generate, slov = False):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs}")
        for phase in ["T","V"]:
            if phase == 'T':
                net.train()  # Установить модель в режим обучения
                dataset=train_dataset
            else:
                net.eval()   #Установить модель в режим оценки
                dataset=val_dataset

            epoch_loss = 0.0
            for i,x in enumerate(dataset):

                # x[0] is class label, x is text
                if slov:
                    leng = len(enc_slov(str(x)))
                else:
                    leng = len(str(x))

                if leng-nchars<10:
                    continue

                text_in, text_out = get_batch(x,nchars=nchars)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'T'):
                    out = net(text_in)
                    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
                    if phase == 'T':
                        loss.backward()
                        optimizer.step()
                    else:
                        # scheduler.step(loss.item())
                        pass
                epoch_loss+=loss.item()
                # if i%100==0:
                #     print(f"{i}/{len(dataset)} Current loss {phase} = {epoch_loss/(i+1)}")
            print(f"Current loss {phase} = {epoch_loss / len(dataset)}")
            if phase == 'V':
                print(generate(net,start=test_str))

# Обучение Simple RNN с посимвольной и по-словной токенизацией


In [17]:
class RNNGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.vocab_size=vocab_size
        self.rnn = torch.nn.RNN(self.vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, self.vocab_size)

    def forward(self, x):
        x = torch.nn.functional.one_hot(x,self.vocab_size).to(torch.float32)
        x,h = self.rnn(x)
        return self.fc(x)

In [18]:
nchars = 100
num_layers=3
test_str_orig = "Я застал его в саду. Он осматривал яблони, обнаженные дыханием осени, и с помощию старого садовника бережно их укутывал теплой соломой. Лицо его изображало спокойствие, здоровье и добродушие."
test_str_slov = "Я застал его в саду. Он осматривал яблони, обнаженные дыханием осени,"
test_str = test_str_orig[:nchars]

#### Обучение Simple RNN с посимвольной токенизацией

In [19]:
net=RNNGenerator(vocab_size,nchars).to(device)
optimizer = torch.optim.Adam(net.parameters(),0.01)
scheduler=lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.9,patience=50)

In [20]:
train(net,train_dataset,train_dataset,50,optimizer, scheduler, get_batch, generate)

Epoch 0/50
Current loss T = 0.6193647138975762
Current loss V = 0.5963792931120675
Я застал его в саду. Он осматривал яблони, обнаженные дыханием осени, и с помощию старого садовника во во во во во во во во во во во во во во во во во во во во во во во во во во во во во во во во во в
Epoch 1/50
Current loss T = 0.5489836250608032
Current loss V = 0.5539496049687669
Я застал его в саду. Он осматривал яблони, обнаженные дыханием осени, и с помощию старого садовника во во во во во во во во во во во во во во во во во во во во во во во во во во во во во во во во во в
Epoch 2/50
Current loss T = 0.5310682544300148
Current loss V = 0.5493123007787241
Я застал его в саду. Он осматривал яблони, обнаженные дыханием осени, и с помощию старого садовника ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва ва в
Epoch 3/50
Current loss T = 0.5198707533580763
Current loss V = 0.5468283727899328
Я застал его в саду. Он осматривал яблони, обнаженные дыханием ос

In [21]:
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start=test_str_slov,temperature=i ,enc=enc)}\n")

--- Temperature = 0.3
Я застал его в саду. Он осматривал яблони, обнаженные дыханием осени, ве ве вена ся на зво ва ве ве ве вал ве сте ва за ве во в ве вева ва ве ва ве ва ва ва вева ва ви ва ва ва ве по вого прена сто стото то ва ва ка Фесто сто ве ве ве ве валава вала ве ва вене вастовы ве во Фене и Фе ве Фе ве ви ва ка ве по сте ва ве во дри ве вом. ве ве ва ве сто ве сто да тва призн

--- Temperature = 0.8
Я застал его в саду. Он осматривал яблони, обнаженные дыханием осени,стодазт ном по. в не Пуго бый Фецогоми исинать вы вуготвареравска ве ру Ф, рего я уготското пой бецы кочам. мнодо язадо навито по н ког тытвала Домноведа и дь ва вецодецазъя на Я я с стенемить ськиругрери до возушко в вельют, Мо доле ва биль в стакарага о валалсколими заватодна си гождерих ка гл Фег

--- Temperature = 1.0
Я застал его в саду. Он осматривал яблони, обнаженные дыханием осени, – выма Фн Кабрабыланотрупею. – – – вать тери ко пемой Пу востск ичто ичкотью, вовен, оринул, вавей. тьс учнарей твето Февы

#### Обучение Simple RNN с пословной токенизацией




In [22]:
nchars = 10
vocab_size = len(vocab_slov)

In [23]:
net=RNNGenerator(vocab_size,nchars).to(device)
optimizer = torch.optim.Adam(net.parameters(),0.01)
scheduler=lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.9,patience=50)

In [24]:
train(net,train_dataset,train_dataset,50,optimizer, scheduler, get_batch_slov, generate_slov, slov = True)

Epoch 0/50
Current loss T = 1.704948453752844
Current loss V = 1.5609122138839584
Я застал его в саду. Он осматривал яблони, обнаженные дыханием осени, и с помощию старого садовника  , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что , что
Epoch 1/50
Current loss T = 1.4730481685281873
Current loss V = 1.422020857130085
Я застал его в саду. Он осматривал яблони, обнаженные дыханием осени, и с помощию старого садовника  . – сказал он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он , что он
Epoch 2/50
Current loss T = 1.36

In [25]:
print(f"Результат\n{generate_slov(net,size=300,start=test_str_slov)}\n")

Результат
Я застал его в саду. Он осматривал яблони, обнаженные дыханием осени, ни одного слова , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять о том , что мне был опять 